In [1]:
from keras.models import load_model
from time import sleep 
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import serial
import numpy as np 
Myserial = serial.Serial('COM4',9600)
f_c = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
d_c = load_model("Emotion_little_vgg.h5")
class_ = ["Angry","Happy", "Neutral", "Sad", "Surprise"]
cap = cv2.VideoCapture(0)

while True:
    ret, frame  = cap.read()
    labels = []
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = f_c.detectMultiScale(gray,1.3,5)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        roi = gray[y:y+h,x:x+w]
        roi = cv2.resize(roi,(48,48),interpolation=cv2.INTER_AREA)
        if np.sum(roi) != 0:
            
            roi = roi.astype("float")/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            
            pred = d_c.predict(roi)[0]
            label = class_[pred.argmax()]
            
            if label == "Neutral":
                Myserial.write(str.encode('W'))
            if label == "Happy":
                Myserial.write(str.encode('G'))
            if label == "Sad":
                Myserial.write(str.encode('B'))
            if label == "Angry":
                Myserial.write(str.encode('R'))
            if label == "Surprise":
                Myserial.write(str.encode('M'))
            label_co = (x,y)
            
            cv2.putText(frame,label,label_co, cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
        else:
            cv2.putText(frame,"No_face_Found",40,60, cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    cv2.imshow("Face_emotion_detector",frame)
    if cv2.waitKey(1) & 0XFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()

Using TensorFlow backend.
